In [1]:
# Get your location... aproximately
import geocoder
def getMyLocation():
    g = geocoder.ip('me')
    return g.latlng

getMyLocation()

[-33.8678, 151.2073]

In [2]:
import requests_cache
import openmeteo_requests
from retry_requests import retry
import pandas as pd


# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('./theory/cache/', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
lat, lng = getMyLocation()
params = {
    "latitude": round(lat, 2),
    "longitude": round(lng, 2),
    "hourly": "temperature_2m",
    "timezone": "Australia/Sydney",
    "past_days": 92,
    "models": "bom_access_global"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
    start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
    end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
    freq = pd.Timedelta(seconds = hourly.Interval()),
    inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe

,date,temperature_2m
0,2024-05-19 14:00:00+00:00,7.437500
1,2024-05-19 15:00:00+00:00,7.637500
2,2024-05-19 16:00:00+00:00,7.937500
3,2024-05-19 17:00:00+00:00,7.937500
4,2024-05-19 18:00:00+00:00,7.637500
...,...,...
2371,2024-08-26 09:00:00+00:00,16.887499
2372,2024-08-26 10:00:00+00:00,15.787499
2373,2024-08-26 11:00:00+00:00,14.787499
2374,2024-08-26 12:00:00+00:00,14.037499
